We are [allowed](https://www.kaggle.com/c/halite/discussion/159283) to use C++ in Halite competion. However, I am not sure how to do that technically. We cannot access external files; we even need to embed binary pretrained weights to the submission python script (many thanks to [Nail](https://www.kaggle.com/c/connectx/discussion/126678) and [higepon](https://www.kaggle.com/c/halite/discussion/164005) for sharing how to do that). This is my attempt; let me know if you know a better way.

1. Write a C/C++ extension module for Python as usual; compile it in this Notebook;
2. convert the module file to a string with base64 and paste it to `submission.py`;
3. decode the string and recover the module file in submission.py;
4. import the module.

## 1. Create the module with C/C++ and compile

We need at least two files:
- `spam.cpp`: code in C/C++
- `setup.py`: script to compile

You can `+Add data` or access internet if you want in this step.

In [ ]:
%%writefile setup.py

from distutils.core import setup, Extension

setup(name='myspam',
      version='0.0.1',
      description='Trivial python module written in C++',
      ext_modules=[
          Extension('myspam', ['spam.cpp'],
                    extra_compile_args = ['-std=c++11'])
      ]
)

In [ ]:
%%writefile spam.cpp

#include <cstdlib>
#include "Python.h"

// C++ funcutions here
PyObject* myrand(PyObject *self, PyObject *args)
{
  return Py_BuildValue("i", rand() % 4);
}

// List the functions accessible from Python
static PyMethodDef methods[] = {
  {"myrand", myrand, METH_VARARGS, "return random integer 0-3"},
  {NULL, NULL, 0, NULL}
};

static struct PyModuleDef module = {
  PyModuleDef_HEAD_INIT,
  "myspam",               // name of this module
  "C++ module example",   // Doc String
  -1,
  methods
};

PyMODINIT_FUNC
PyInit_myspam(void) {  
  return PyModule_Create(&module);
}


### Compile

In [ ]:
!python3 setup.py build_ext --inplace

We can find `myspam.cpython-37m-x86_64-linux-gnu.so`.

In [ ]:
! ls

In [ ]:
# You can use the module in the notebook like this,
# but submission.py probably cannot access this file
import myspam
myspam.myrand()

## 2. Convert the module to a string and embed in submission.py


In [ ]:
import base64

with open('myspam.cpython-37m-x86_64-linux-gnu.so', 'rb') as f:
    encoded_string = base64.b64encode(f.read())

with open('submission.py', 'w') as f:
    f.write(f'module_str={encoded_string}')

## 3. In submission.py

- Decode the module file,
- write it to file,
- and import

In [ ]:
%%writefile -a submission.py

import base64
import kaggle_environments.envs.halite.helpers as hh
with open('myspam.cpython-37m-x86_64-linux-gnu.so', 'wb') as f:
    f.write(base64.b64decode(module_str))
import myspam

actions = [hh.ShipAction.NORTH, hh.ShipAction.EAST,
           hh.ShipAction.SOUTH, hh.ShipAction.WEST]

# Trivial Halite agent randomly walking
def agent(obs, config):
    board = hh.Board(obs, config)
    me = board.current_player

    for ship in me.ships:
        i = myspam.myrand()  # Use the C++ code
        ship.next_action = actions[i]
    
    return me.next_actions

## 4. Submit submission.py

The agent is superweak, of course, but it works.